In [ ]:
import geopandas as gpd
import pandas as pd
from pyproj import Proj, CRS, Transformer
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium import plugins
import branca as cm
import seaborn as sns
plt.rcParams['figure.figsize'] = [8,8]

In [ ]:
stream=gpd.read_file("../geojson/stream/regionalwatersicp.geojson")

In [ ]:
print(len(stream))
stream.columns

In [ ]:
stream_contaminants=stream[["AS_","CD","CR","CU","NI","P","PB","ZN","geometry"]].copy()
stream_contaminants.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

In [ ]:
a = pd.to_numeric(stream_contaminants["ZN"], errors='coerce')
idx=a.isna()
stream_contaminants["ZN"][idx].to_frame()

In [ ]:
stream_contaminants["ZN"]=stream_contaminants["ZN"].fillna(0)
stream_contaminants["P"]=stream_contaminants["P"].fillna(0)
stream_contaminants["P"].replace(',','', regex=True, inplace=True)
stream_contaminants["P"]=pd.to_numeric(stream_contaminants["P"])
stream_contaminants.info()

In [ ]:
sum(i>5 for i in stream_contaminants["ZN"])

In [ ]:
stream_contaminants["ZN"].describe()


In [ ]:
stream_contaminants['lat']=stream_contaminants.geometry.y
stream_contaminants['lng']=stream_contaminants.geometry.x

stream_P=stream_contaminants.loc[(stream_contaminants['P'] > 50)]
stream_AS=stream_contaminants.loc[(stream_contaminants['AS_'] > 10)]
stream_CD=stream_contaminants.loc[(stream_contaminants['CD'] > 5)]
stream_PB=stream_contaminants.loc[(stream_contaminants['PB'] > 10)]
stream_CU=stream_contaminants.loc[(stream_contaminants['CU'] > 2)]
stream_CR=stream_contaminants.loc[(stream_contaminants['CR'] > 50)]
stream_NI=stream_contaminants.loc[(stream_contaminants['NI'] > 20)]
stream_ZN=stream_contaminants.loc[(stream_contaminants['ZN'] > 5)]

In [ ]:
'''
fig = folium.Figure(width=1200, height=1000)
colormap_P = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,50,13000])
colormap_AS = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,10,46])
colormap_CD = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,5,6])
colormap_PB = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,10,45])
colormap_CU= cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,2,188])
colormap_CR = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,50,55])
colormap_NI = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,20,69])
colormap_ZN = cm.colormap.LinearColormap(colors=["lightgray", "red"]).to_step(index=[0,5,364])

m=stream_contaminants.explore(column="P", cmap = colormap_P, show=False, legend=False, name="Potassium", marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="AS_", cmap = colormap_AS,show=False, legend=False, name="Arsenic", marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="CD", cmap = colormap_CD, show=False, legend=False, name="Cadmium",  marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="PB", cmap = colormap_PB, show=False, legend=False, name="Lead", marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="CU", cmap = colormap_CU, show=False, legend=False, name="Copper", marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="CR", cmap = colormap_CR,show=False, legend=False, name="Chromium",  marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="NI", cmap = colormap_NI, show=False, legend=False, name="Nickel", marker_kwds={"radius": 3})
stream_contaminants.explore(m=m, column="ZN", cmap = colormap_ZN, show=False, legend=False, name="Zinc", marker_kwds={"radius": 3})


folium.TileLayer('stamenterrain', name="Terrain").add_to(m)
folium.TileLayer('cartodbpositron', name="Greyscale").add_to(m)
folium.TileLayer('cartodbdark_matter', name="Dark").add_to(m)
plugins.Geocoder().add_to(m)
folium.LayerControl(collapsed=False).add_to(m)  # use folium to add layer control


m.add_to(fig)

''';

In [ ]:
def create_nimap():
    nimap = folium.Map(
        location=[54.637521, -6.485935],
        zoom_start=9,
    )
    return nimap

In [ ]:
m = create_nimap()
fig = folium.Figure(width=1200, height=1000)

group_1 = folium.FeatureGroup("Phosphorous", show=False).add_to(m)
for loc in zip(stream_P["lat"],stream_P["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color="rebeccapurple",
                          fill_opacity=1
                          ).add_to(group_1)


group_2 = folium.FeatureGroup("Arsenic", show = False).add_to(m)
for loc in zip(stream_AS["lat"],stream_AS["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color="blue",
                          fill_opacity=1
                          ).add_to(group_2)

group_3 = folium.FeatureGroup("Cadmium", show=False).add_to(m)
for loc in zip(stream_CD["lat"],stream_CD["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color="green",
                          fill_opacity=1
                          ).add_to(group_3)

group_4 = folium.FeatureGroup("Lead", show=False).add_to(m)
for loc in zip(stream_PB["lat"],stream_PB["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 5,
                          fill=True,
                          color="red",
                          fill_opacity=1
                          ).add_to(group_4)

group_5 = folium.FeatureGroup("Copper", show=False).add_to(m)
for loc in zip(stream_CU["lat"],stream_CU["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color="gray",
                          fill_opacity=1
                          ).add_to(group_5)

group_6 = folium.FeatureGroup("Chromium",show=False).add_to(m)
for loc in zip(stream_CR["lat"],stream_CR["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color="magenta",
                          fill_opacity=1
                          ).add_to(group_6)

group_7 = folium.FeatureGroup("Nickel", show=False).add_to(m)
for loc in zip(stream_NI["lat"],stream_NI["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 5,
                          fill=True,
                          color="limegreen",
                          fill_opacity=1
                          ).add_to(group_7)

group_8 = folium.FeatureGroup("Zinc", show=False).add_to(m)
for loc in zip(stream_ZN["lat"],stream_ZN["lng"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color="orange",
                          fill_opacity=1
                          ).add_to(group_8)

folium.TileLayer('stamenterrain', name="Terrain").add_to(m)
folium.TileLayer('cartodbpositron', name="Greyscale").add_to(m)
folium.TileLayer('cartodbdark_matter', name="Dark").add_to(m)
plugins.Geocoder().add_to(m)
folium.LayerControl(collapsed = False).add_to(m)


#map_title = 'Corpus Christi'
#title_html = '''
#             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
#             '''.format(map_title)   
#
#m.get_root().html.add_child(folium.Element(title_html))

m.add_to(fig)